In [1]:
# %load_ext autoreload
# %autoreload 2
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithHuman

gms8k = GSM8K()
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)

trainset, devset = gms8k.train, gms8k.dev

dspy.settings.configure(lm=turbo)

/Users/ben/code/dspy/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1319/1319 [00:00<00:00, 93299.50it/s]


In [2]:
import argilla as rg

client = rg.Argilla(api_key="argilla.apikey", api_url="http://localhost:6900")

dataset_name = "gsm8k"
if client.datasets(dataset_name):
    client.datasets(dataset_name).delete()

rg_dataset = rg.Dataset(
    name=dataset_name,
    settings=rg.Settings(
        fields=[
            rg.TextField(name="question"),
        ],
        questions=[
            rg.TextQuestion(
                name="answer",
            )
        ],
    ),
)

rg_dataset.create()

rg_dataset.records.log(
    [dict(example) for example in trainset], mapping={"answer": "answer.response"}
)

/Users/ben/code/dspy/.venv/lib/python3.11/site-packages/argilla/datasets/_resource.py:264: UserWarning: Workspace not provided. Using default workspace: argilla id: 735cae0d-eb08-45c3-ad79-0a11ad4dd2c2
  warnings.warn(f"Workspace not provided. Using default workspace: {workspace.name} id: {workspace.id}")
/Users/ben/code/dspy/.venv/lib/python3.11/site-packages/argilla/records/_mapping/_mapper.py:89: UserWarning: Keys ['gold_reasoning'] in data are not present in the mapping and will be ignored.
  warnings.warn(f"Keys {unknown_keys} in data are not present in the mapping and will be ignored.")


DatasetRecords: The provided batch size 256 was normalized. Using value 200.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  1.95batch/s]


DatasetRecords(Dataset(id=UUID('6b884925-0c35-4b77-b1f8-1132458a39bf') inserted_at=datetime.datetime(2024, 10, 17, 19, 29, 44, 363607) updated_at=datetime.datetime(2024, 10, 17, 19, 29, 44, 507762) name='gsm8k' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('735cae0d-eb08-45c3-ad79-0a11ad4dd2c2') last_activity_at=datetime.datetime(2024, 10, 17, 19, 29, 44, 507762)))

In [3]:
NUM_THREADS = 4
evaluate = Evaluate(
    devset=devset[:],
    metric=gsm8k_metric,
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=0,
)

In [4]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(signature="question -> answer")

    def forward(self, question):
        return self.prog(question=question)

In [6]:
teleprompter = BootstrapFewShotWithHuman(
    metric=gsm8k_metric,
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    num_candidate_programs=2,
    num_threads=NUM_THREADS,
    argilla_dataset=rg_dataset,
)

cot_bs = teleprompter.compile(
    student=CoT(),
    trainset=trainset[:10],
    valset=devset[:10],
)

 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 2 candidate sets.


 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
  0%|          | 0/10 [00:00<?, ?it/s]WARNING:root:	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated. ***
 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
Average Metric: 6 / 10  (60.0): 100%|██████████| 10/10 [00:00<00:00, 662.90it/s]


New best score: 60.0 for seed -3
Scores so far: [60.0]
Best score so far: 60.0


Average Metric: 6 / 10  (60.0): 100%|██████████| 10/10 [00:00<00:00, 1298.71it/s]


...

Scores so far: [60.0, 60.0, 80.0, 80.0]
Best score so far: 80.0


 40%|████      | 4/10 [00:00<00:00, 2029.42it/s]


Bootstrapped 3 full traces after 5 examples in round 0.


Average Metric: 9 / 10  (90.0): 100%|██████████| 10/10 [00:00<00:00, 1799.90it/s]

New best score: 90.0 for seed 1
Scores so far: [60.0, 60.0, 80.0, 80.0, 90.0]
Best score so far: 90.0
5 candidate programs found.


In [7]:
evaluate(cot_bs, devset=devset[:20])

Average Metric: 14 / 20  (70.0): 100%|██████████| 20/20 [00:00<00:00, 1704.62it/s]


70.0

In [7]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Mark is baking bread. He has to let it rise for 120 minutes twice. He also needs to spend 10 minutes kneading it and 30 minutes baking it. How many minutes does it take Mark to finish making the bread?
Reasoning: Let's think step by step in order to find the total time it takes Mark to finish making the bread. We know that he has to let the bread rise for 120 minutes twice, which is 240 minutes. He also needs to spend 10 minutes kneading it and 30 minutes baking it. Therefore, the total time it takes Mark to finish making the bread is 240 + 10 + 30 = 280 minutes.
Answer: 280

---

Question: Ben has $2000 for his business operations costs. He orders goods from his supplier and writes them a cheque for $600. His debtor pays him $800 from the purchases they had 

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Mark is baking bread. He has to let it rise for 120 minutes twice. He also needs to spend 10 minutes kneading it and 30 minutes baking it. How many minutes does it take Mark to finish making the bread?\nReasoning: Let's think step by step in order to find the total time it takes Mark to finish making the bread. We know that he has to let the bread rise for 120 minutes twice, which is 240 minutes. He also needs to spend 10 minutes kneading it and 30 minutes baking it. Therefore, the total time it takes Mark to finish making the bread is 240 + 10 + 30 = 280 minutes.\nAnswer: 280\n\n---\n\nQuestion: Ben has $2000 for his business operations costs. He orders goods from his supplier and writes them a cheque for $600. His debtor pays him $800 from t